# Nearest Neighbor Approach

## Preparing the data

In [ ]:
pip install ortools

In [ ]:
pip install openrouteservice

In [3]:
from ortools.constraint_solver import pywrapcp, routing_enums_pb2
import openrouteservice
import time
import pandas as pd
import numpy as np
import json
import math

In [5]:
with open("data.json") as f:
  raw_data = json.load(f)

#raw_data

In [6]:
nodes = []

for entry in raw_data:
    node_id = entry["CustomerID"]
    x = entry["Longitude"]
    y = entry["Latitude"]
    city = entry["City"]
    demand = entry["Demand"]
    net_profit = entry["Net profit"]


    nodes.append({
        'node_id': node_id,
        'x': x,
        'y': y,
        'city': city,
        'demand': demand,
        'net_profit': net_profit
    })

In [ ]:
nodes # customerID = ID_0 is the depot

In the next code chunk, we will compute the distances using **OpenStreetMap (OSM)**, by adopting the routing service **OSRM (Open Source Routing Machine)**.

In [ ]:
client = openrouteservice.Client(key='insert_your_API_key_here')

# Initializing a list of coordinates
#coordinates = [(n['x'], n['y']) for n in virtual_nodes]
locations = [[node['x'], node['y']] for node in nodes]

# Distance matrix
num_nodes = len(locations)
chunk_size = 10
full_matrix = np.zeros((num_nodes, num_nodes))

for i in range(0, num_nodes, chunk_size):
    for j in range(0, num_nodes, chunk_size):
        sources = list(range(i, min(i + chunk_size, num_nodes)))
        destinations = list(range(j, min(j + chunk_size, num_nodes)))

        try:
            response = client.distance_matrix(
                locations=locations,
                profile='driving-hgv', # keeps into account heavy goods vehicles restrictions
                metrics=['distance'],
                units='km',
                resolve_locations=False,
                sources=sources,
                destinations=destinations
            )

            distances = response['distances']
            for si, src in enumerate(sources):
                for di, dst in enumerate(destinations):
                    full_matrix[src][dst] = distances[si][di]

            print(f"Processed block: rows {i}-{i+len(sources)-1}, cols {j}-{j+len(destinations)-1}")

        except Exception as e:
            print(f"Error on block ({i},{j}): {e}")
            print("Sleeping to avoid hitting rate limit...")
            time.sleep(60)  # wait a minute before retrying or continuing

In [9]:
node_ids = [node['node_id'] for node in nodes]

distance_matrix_km = pd.DataFrame(full_matrix, index=node_ids, columns=node_ids)
#print(distance_matrix_km)
csv_output_path = '/content/distance_matrix_km.csv'
distance_matrix_km.to_csv(csv_output_path)

In [10]:
distance_matrix_df = pd.read_csv('/content/distance_matrix_km.csv', index_col=0)
distance_matrix_df.head()

,ID_0,ID_1,ID_2,ID_3,ID_4,ID_5,ID_6,ID_7,ID_8,ID_9,...,ID_11,ID_12,ID_13,ID_14,ID_15,ID_16,ID_17,ID_18,ID_19,ID_20
ID_0,0.00,68.64,134.81,9.26,127.81,11.38,22.35,178.42,4.24,39.38,...,39.38,36.30,158.99,181.51,173.98,157.26,175.74,180.13,138.43,72.16
ID_1,69.42,0.00,83.56,83.90,76.56,87.99,54.02,127.16,69.87,87.64,...,87.65,101.89,107.74,130.26,122.73,106.01,124.49,128.88,87.17,94.75
ID_2,137.21,82.11,0.00,151.69,12.71,146.55,124.35,50.43,140.20,116.88,...,116.89,169.68,31.01,53.52,45.99,29.27,47.75,52.14,27.53,74.83
ID_3,9.26,83.07,149.24,0.00,142.24,4.59,29.18,192.85,11.07,46.59,...,46.59,32.21,173.43,195.94,188.41,171.69,190.17,194.56,152.86,82.27
ID_4,129.95,74.85,12.75,144.43,0.00,139.29,117.09,61.12,132.94,109.63,...,109.63,162.43,41.70,64.22,56.69,39.97,58.44,62.84,18.95,67.57


In [12]:
distance_matrix = distance_matrix_df.to_numpy()
#print(distance_matrix)

In [13]:
# Checking if the distances stored in the matrix are correct

from_node = 1 # row (ID)
to_node = 0 # column (ID)

coords = [locations[from_node], locations[to_node]]
route = client.directions(coords, profile='driving-hgv', units='km')
print(f"ORS route distance: {route['routes'][0]['summary']['distance']} km")

# The distances are slightly asymmetric

ORS route distance: 69.419 km


## Setting up the model

We are defining a **closed multi-vehicle routing problem (CVRP)**, where our goal is to determine the optimal route to serve all customers using the shortest distance. Since the problem is NP-hard, we will use heuristics.

When a customer's demand exceeds any vehicle's capacity, OR-Tools cannot assign that node to any truck under its default "visit once" constraint. To fix this, we need to:
* split the single high-demand nodes into multiple **virtual nodes**, each with a smaller demand
* each virtual node represents a partial delivery to the same location
* these virtual nodes allow OR-Tools to serve the customer with multiple trucks, each within capacity limits.

Therefore, we filter the nodes, so that the ones having a demand that exceeds the maximum capacity are split into sub-nodes.

In [14]:
allowed_capacity = [26, 34]

vehicles = 60 # number of trucks

In [15]:
def split_nodes(nodes, allowed_capacity):
    allowed_capacity = sorted(allowed_capacity)

    if set(allowed_capacity) != {26, 34}:
        raise ValueError("The capacity must be either 26 or 34]")

    min_cap, max_cap = allowed_capacity # min_cap, max_cap = 26, 34
    split_nodes = []

    for node in nodes:
        demand = node["demand"]
        if demand == 0: # skip depot
            continue

        base_profit = node["net_profit"] / demand

        if demand <= min_cap:
            split_nodes.append(node)
        elif demand <= max_cap:
            split_nodes.append(node)
        else:
            parts = []
            remaining = demand # tracking part of the demand that still need to be split

            while remaining > max_cap:
                parts.append(max_cap)
                remaining -= max_cap

            if remaining > 0:
              parts.append(remaining)

            for i, part_demand in enumerate(parts):
                sub_node = {
                    'node_id': f"{node['node_id']}_{i+1}",
                    'x': node['x'],
                    'y': node['y'],
                    'city': node['city'],
                    'demand': part_demand,
                    'net_profit': round(part_demand * base_profit, 2)
                }
                split_nodes.append(sub_node)

    return split_nodes

In [16]:
split_nodes = split_nodes(nodes, allowed_capacity)
#split_nodes
#len(split_nodes) --> 54

In [17]:
virtual_nodes = [nodes[0]] + split_nodes # adding the depot
# len(virtual_nodes) --> 55

In [ ]:
#virtual_nodes

In [18]:
# Map original node IDs to their indices in the original distance matrix
original_index_map = {node['node_id']: idx for idx, node in enumerate(nodes)}

# Extract virtual ID from original ID
def extract_original_id(virtual_id):
    return '_'.join(virtual_id.split('_')[:2])

# Map each virtual node to its original node
virtual_to_original = {
    v_node['node_id']: extract_original_id(v_node['node_id'])
    for v_node in virtual_nodes
}

# Prepare new virtual distance matrix
n = len(virtual_nodes)
virtual_distance_matrix = np.zeros((n, n))
epsilon = 0.001  # small nonzero distance to avoid loops

'''
For example, if the original node has index 1 in the original distance matrix (full_matrix),
then all of its sub-nodes (virtual nodes) will refer to that same index when building the virtual_distance_matrix.
'''
for i, vi in enumerate(virtual_nodes):
    oi_idx = original_index_map[virtual_to_original[vi['node_id']]]

    for j, vj in enumerate(virtual_nodes):
        oj_idx = original_index_map[virtual_to_original[vj['node_id']]]

        if i == j:
            virtual_distance_matrix[i][j] = 0
        elif virtual_to_original[vi['node_id']] == virtual_to_original[vj['node_id']]:
            virtual_distance_matrix[i][j] = epsilon
        else:
            virtual_distance_matrix[i][j] = full_matrix[oi_idx][oj_idx]


In [19]:
virtual_distance_matrix = np.where(
    (virtual_distance_matrix % 1) > 0.5, # rounding if the decimal > 0.5
    np.ceil(virtual_distance_matrix),
    np.floor(virtual_distance_matrix)
).astype(int)

In [20]:
#virtual_distance_matrix

In [21]:
def create_model():
    data = {}
    data['virtual_distance_matrix'] = virtual_distance_matrix
    data['demand'] = [node['demand'] for node in virtual_nodes]
    data['net_profit'] = [node['net_profit'] for node in virtual_nodes]
    '''
    OR-Tools doesn't natively support vehicles returning to the depot and starting again.
    '''
    data['vehicles'] = vehicles # number of trucks
    data['vehicles_capacity'] = [34] * 45 + [26] * 15 # 26 = small trucks, 34 = big trucks
    '''
    The capacity for each truck can vary (one pallet = 1000 bottles),
    depending on the dimensions of the vehicle. Smaller trucks can carry up to 26 pallets
    while larger ones can carry up to 34 pallets.
    '''
    data['depot'] = 0  # node at index 0 = "ID_0", that is the depot
    return data

data = create_model()

'''
Manager handles mapping between our problem data and OR-Tool's internal index system
'''
manager = pywrapcp.RoutingIndexManager(
    len(data['virtual_distance_matrix']),
    data['vehicles'],
    data['depot']
    )

'''
Actual solver model
'''
routing = pywrapcp.RoutingModel(manager)

In [22]:
overall_profit = sum(data['net_profit'])
#overall_profit

In [ ]:
#data

In [ ]:
# Debugging
print(len(data['virtual_distance_matrix'])) # must be 55 = length of virtual nodes
print(len(data['demand'])) # must be 55
print(len(data['net_profit'])) # must be 55
print(data['demand'][0])
print(data['vehicles']) # 60
print(len(data['vehicles_capacity'])) # [26,34]

### Defining the objective function

First of all, we need to define the **distance callback**, that is a function that takes any pair of locations and returns the distance between them.

After, we need to define the **arc cost evaluator** which tells the solver how to compute the distance between two nodes. This means that the cost of travel between two locations is just the distance between them.

In [ ]:
'''
  Takes a pair of locations  as input. IndexToNode translates OR-Tools' internal
  routing indices back into our original virtual nodes indices to compute the
  distance between the two locations
'''

def distance_callback(from_index, to_index):
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return int(data["virtual_distance_matrix"][from_node][to_node])

transit_callback_index = routing.RegisterTransitCallback(distance_callback)

# Setting arc cost evaluator
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

In [ ]:
# Checking...
test_value = distance_callback(0, 1)
print("Manual distance_function(0, 1):", test_value) # correct

### Adding the capacity constraint

Beside the distance callback, we also need to define the **demand callback** that returns the demand at each location, and a dimension for the capacity constraint.

Additionally, `AddDimensionsWithVehicleCapacity` takes a vector of capacities.

In [ ]:
# Adding the capacity constraint (demand callback)

def demand_callback(from_index):
    from_node = manager.IndexToNode(from_index) # only depends on from_node, unlike the distance callback
    return data["demand"][from_node]

demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)

routing.AddDimensionWithVehicleCapacity(
    demand_callback_index,
    0,  # vehicles loads must strictly not exceed their capacity --> no slack
    data["vehicles_capacity"],  # max capacity for each vehicle
    True, # --> manually set the initial load in the "Displaying the solutions" code chunk
    "Capacity"
)

In [ ]:
# Check if the total demand exceeds the capacity constraint

total_demand = sum(data['demand'])
total_capacity = sum(data['vehicles_capacity'])
print(f"Total Demand: {total_demand}, Total Capacity: {total_capacity}") # Yes

Since we are considering a **closed multi-vehicle routing problem**, we need to ensure that our model doesn't generate loops or suboptimal paths that don't return to the depot. *Subtour elimination* is handled automatically by OR-Tools through the *flow conservation constraint*, which ensures that:
* all vehicles return to the depot after completing the assigned routes
* multi-trips are not allowed, meaning that the trucks can't return to the depot to "reload"


## Solving the problem

In this section, we define the search parameters to perform the constructive and improving heuristic approaches.

* `Guided Local Search` does not guarantee a global or even a local optimum in the strict mathematical sense, but it is designed to help escape poor local optima and find better solutions compared to basic local search.

In [24]:
# First: Constructive approach (PATH_CHEAPEST_ARC)

search_parameters_first = pywrapcp.DefaultRoutingSearchParameters()
search_parameters_first.time_limit.FromSeconds(30)

search_parameters_first.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

solution_first = routing.SolveWithParameters(search_parameters_first)

In [25]:
# Second: Improving approach (GUIDED_LOCAL_SEARCH)

search_parameters_second = pywrapcp.DefaultRoutingSearchParameters()
search_parameters_second.time_limit.FromSeconds(30)

search_parameters_second.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
search_parameters_second.local_search_metaheuristic = (routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)

solution_second = routing.SolveWithParameters(search_parameters_second)

## Displaying the solutions

In [ ]:
# Checking if the distances are non-zero...

total_distance = 0
for vehicle_id in range(data['vehicles']):
    index = routing.Start(vehicle_id)
    route_distance = 0
    route_nodes = []
    while not routing.IsEnd(index):
        route_nodes.append(manager.IndexToNode(index))
        next_index = solution_first.Value(routing.NextVar(index)) # solution_second
        route_distance += routing.GetArcCostForVehicle(index, next_index, vehicle_id)
        index = next_index
    route_nodes.append(manager.IndexToNode(index))
    print(f"Vehicle {vehicle_id} route: {route_nodes} distance: {route_distance} km")
    total_distance += route_distance
print(f"Total distance of all routes: {total_distance} km")


In [ ]:
def print_solution(data, manager, routing, solution):
    print(f"Objective: {solution.ObjectiveValue()}") # ObjectiveValue() returns the optimal route and distance
    total_distance = 0
    total_load = 0

    for vehicle_id in range(data["vehicles"]):
        index = routing.Start(vehicle_id)
        route_load = data["vehicles_capacity"][vehicle_id]  # start full
        plan_output = f"Route for vehicle {vehicle_id}:\n"
        route_distance = 0

        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            plan_output += f" {node_index} Load({route_load}) -> "
            previous_index = index
            index = solution.Value(routing.NextVar(index))

            if node_index != 0: # skipping the depot
                route_load -= data["demand"][node_index]

            route_distance += routing.GetArcCostForVehicle(previous_index, index, vehicle_id)

        plan_output += f" {manager.IndexToNode(index)} Load({route_load})\n"
        plan_output += f"Distance of the route: {route_distance}km\n"
        delivered_load = data['vehicles_capacity'][vehicle_id] - route_load
        plan_output += f"Amount delivered: {delivered_load}\n"
        print(plan_output)

        total_distance += route_distance
        total_load += delivered_load

    print(f"Total distance of all routes: {total_distance}km")
    print(f"Total amount delivered: {total_load}")

In [ ]:
# First solution
print_solution(data,manager,routing,solution_first) # 0 = truck in excess

In [ ]:
# Second solution
print_solution(data,manager,routing,solution_second) # 0 = truck in excess

## Visualization in QGIS

In [ ]:
pip install geojson

In [ ]:
node_ids = [vn['node_id'] for vn in virtual_nodes]

node_coords = {i: (vn['x'], vn['y']) for i, vn in enumerate(virtual_nodes)}

# Mapping
index_to_node_id = {i: vn['node_id'] for i, vn in enumerate(virtual_nodes)}
node_id_to_index = {vn['node_id']: i for i, vn in enumerate(virtual_nodes)}

In [ ]:
# Generating a list of routes for each solution

def extract_routes(manager, routing, solution):
    all_routes = []
    for vehicle_id in range(routing.vehicles()):
        index = routing.Start(vehicle_id)
        route = []
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route.append(node_index)
            index = solution.Value(routing.NextVar(index))
        route.append(manager.IndexToNode(index))
        if len(route) > 2:  # skip unused vehicles
            all_routes.append(route)
    return all_routes

routes_first = extract_routes(manager, routing, solution_first)
routes_second = extract_routes(manager, routing, solution_second)

In [ ]:
coord_list = list(node_coords.values())

In [ ]:
import geojson
'''
def routes_to_geojson(routes, node_coords, label_prefix):
    features = []
    for idx, route in enumerate(routes):
        coords = [node_coords[node] for node in route]
        line = geojson.LineString(coords)
        feature = geojson.Feature(geometry=line, properties={"route": f"{label_prefix}_{idx}"})
        features.append(feature)
    return geojson.FeatureCollection(features)
'''

def get_route_geometry(client, coord_list): # client = API key
    route = client.directions(
        coordinates=coord_list,
        profile='driving-hgv', # '-hgv' for heavy goods vehicles
        format='geojson',
        radiuses=[1000] * len(coord_list) # must be of same length of coord_list
    )
    return route['features'][0]['geometry']


In [ ]:
def routes_to_geojson(routes, node_coords, label_prefix, client):
    features = []
    for idx, route in enumerate(routes):
        coords = [node_coords[node] for node in route]
        try:
            geometry = get_route_geometry(client, coords)
            feature = geojson.Feature(geometry=geometry, properties={"route": f"{label_prefix}_{idx}"})
            features.append(feature)
        except Exception as e:
            print(f"Error in generating route for {label_prefix}_{idx}: {e}")
    return geojson.FeatureCollection(features)


In [ ]:
geojson_first = routes_to_geojson(routes_first, node_coords, "First", client)
geojson_second = routes_to_geojson(routes_second, node_coords, "Second", client)

with open("first_solution_routes.geojson", "w") as f:
    geojson.dump(geojson_first, f)

with open("second_solution_routes.geojson", "w") as f:
    geojson.dump(geojson_second, f)


# Metaheuristic Tuning

In this section, we're going to compare different solutions obtained using various OR-Tools solvers.

So far, we have used `GUIDED_LOCAL_SEARCH` which is designed to escape the local minima by encouraging the solver to explore alternatives. It's particularly effective since it balaces eploration (trying new solutions) and exploitation (improving exisiting solutions), making it suitable in constraint-heavy problems such as CVRPs.

Most importantly, it respects hard constraints such as capacity, and attempts to include all the available nodes if possible.


## Third solution

`TABU_SEARCH` may skip certain moves which might unintentionally block needed routes necessary to visit all nodes. Since OR-Tools, by default, does not force the solver to serve all customers, `TABU_SEARCH` may provide a partial solution if no feasible improvement exists.

In [ ]:
# Third: Improving approach (TABU_SEARCH)
search_parameters_third = pywrapcp.DefaultRoutingSearchParameters()
search_parameters_third.time_limit.FromSeconds(30)

search_parameters_third.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
search_parameters_third.local_search_metaheuristic = (routing_enums_pb2.LocalSearchMetaheuristic.TABU_SEARCH)

solution_third = routing.SolveWithParameters(search_parameters_third)

In [ ]:
# Third solution
print_solution(data,manager,routing,solution_third) # 0 = truck in excess

# Same solution as the original local search

## Fourth solution

In [ ]:
# Fourth: Improving approach (AUTOMATIC)
search_parameters_fourth = pywrapcp.DefaultRoutingSearchParameters()
search_parameters_fourth.time_limit.FromSeconds(30)

#search_parameters_fourth.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.AUTOMATIC) > weak heuristic
search_parameters_fourth.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
search_parameters_fourth.local_search_metaheuristic = (routing_enums_pb2.LocalSearchMetaheuristic.AUTOMATIC)

solution_fourth = routing.SolveWithParameters(search_parameters_fourth)

In [ ]:
# Fourth solution
print_solution(data,manager,routing,solution_fourth) # 0 = truck in excess

# Same solution as the First Solution

## Fifth solution

In [ ]:
# Fifth: Improving approach (SIMULATED_ANNEALING)
search_parameters_fifth = pywrapcp.DefaultRoutingSearchParameters()
search_parameters_fifth.time_limit.FromSeconds(30)

search_parameters_fifth.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
search_parameters_fifth.local_search_metaheuristic = (routing_enums_pb2.LocalSearchMetaheuristic.SIMULATED_ANNEALING)

solution_fifth = routing.SolveWithParameters(search_parameters_fifth)

In [ ]:
# Fifth solution
print_solution(data,manager,routing,solution_fifth) # 0 = truck in excess

# Same solution as the First Solution

## Sixth solution

In [ ]:
# Fifth: Improving approach (GREEDY_DESCENT)
search_parameters_sixth = pywrapcp.DefaultRoutingSearchParameters()
search_parameters_sixth.time_limit.FromSeconds(30)

search_parameters_sixth.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
search_parameters_sixth.local_search_metaheuristic = (routing_enums_pb2.LocalSearchMetaheuristic.GREEDY_DESCENT)

solution_sixth = routing.SolveWithParameters(search_parameters_sixth)

In [ ]:
# Sixth solution
print_solution(data,manager,routing,solution_sixth) # 0 = truck in excess

# Same solution as the First Solution

# Cluster-first, Route-second Approach

This approach entails partitioning the customers (nodes) into clusters. The goal is to divide the entire customer set into smaller subsets. Clustering can be based on different criteria, such as geographic proximity or demand similarity.

After clustering, we solve a routing problem separately for each cluster. For each of them, we find the optimal or near-optimal route that starts and ends at the depot and visits all customers in that cluster.


In [ ]:
node_ids = [node['node_id'] for node in virtual_nodes]
node_id_to_index = {vn['node_id']: i for i, vn in enumerate(virtual_nodes)}
demands = data['demand']
vehicles_capacity = data['vehicles_capacity']

## Clustering by closeness

### Clustering

The following algorithm picks the first unvisited node as a seed for a cluster, then finds all the other unvisited nodes and sorts them by distance from this seed. After, adds the closest node to the cluster.

In [ ]:
def cluster_by_closeness(demands, virtual_distance_matrix, max_cluster_size=9):
    """
    max_cluster_size: optional max number of nodes per cluster to avoid huge clusters.
    """
    n = len(demands)
    visited = [False] * n
    visited[0] = True  # depot excluded from clustering --> depot is not a node
    clusters = []

    while any(not v for v in visited[1:]):
        candidates = [i for i in range(1, n) if not visited[i]]
        seed = candidates[0]  # picking the first unvisited node

        cluster = [seed]
        visited[seed] = True

        virtual_distances = [(virtual_distance_matrix[seed][j], j) for j in range(1, n) if not visited[j]]
        virtual_distances.sort()

        for _, node in virtual_distances:
            if max_cluster_size and len(cluster) >= max_cluster_size:
                break
            cluster.append(node)
            visited[node] = True

        clusters.append(cluster)

    return clusters

### Solving the problem

In [ ]:
def solve_clusters(clusters, virtual_distance_matrix, demands, vehicles_capacity, vehicles, use_local_search=True):
    from ortools.constraint_solver import pywrapcp, routing_enums_pb2

    depot = 0
    num_nodes = len(demands)

    data = {
        'virtual_distance_matrix': virtual_distance_matrix,
        'demand': demands,
        'vehicles_capacity': vehicles_capacity,
        'vehicles': vehicles,
        'depot': depot
    }

    manager = pywrapcp.RoutingIndexManager(
        len(data['virtual_distance_matrix']),
        data['vehicles'],
        data['depot']
    )

    routing = pywrapcp.RoutingModel(manager)

    # Distance callback
    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['virtual_distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Demand callback
    def demand_callback(from_index):
        from_node = manager.IndexToNode(from_index)
        return data['demand'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # no slack
        data['vehicles_capacity'],
        True,
        'Capacity'
    )

    # Map each node to a cluster
    node_to_cluster = {}
    for cluster_idx, cluster_nodes in enumerate(clusters):
        for node in cluster_nodes:
            node_to_cluster[node] = cluster_idx
    node_to_cluster[depot] = -1  # depot not in any cluster

    # Assign vehicles to clusters
    vehicles_per_cluster = vehicles // len(clusters)
    node_allowed_vehicles = {node: set(range(vehicles)) for node in range(num_nodes)}

    for cluster_idx, cluster_nodes in enumerate(clusters):
        cluster_vehicles = range(cluster_idx * vehicles_per_cluster, (cluster_idx + 1) * vehicles_per_cluster)

        for node in range(1, num_nodes):  # exclude depot
            node_cluster = node_to_cluster.get(node, -1)
            if node_cluster != cluster_idx:
                # this node should not be visited by vehicles in this cluster
                for v in cluster_vehicles:
                    if v in node_allowed_vehicles[node]:
                        node_allowed_vehicles[node].remove(v)


    # Search strategy
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.time_limit.seconds = 10
    '''
    First solution --> Constructive approach
    '''
    search_parameters.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    '''
    Local Search --> Improving approach
    '''
    search_parameters.local_search_metaheuristic = routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH

    solution = routing.SolveWithParameters(search_parameters)

    return routing, manager, solution



### Displaying the solution

In [ ]:
clusters = cluster_by_closeness(demands, virtual_distance_matrix, max_cluster_size=9)

routing, manager, solution = solve_clusters(
    clusters,
    virtual_distance_matrix,
    demands,
    vehicles_capacity,
    vehicles,
    use_local_search=True
)

if not solution:
    print("\n No valid solution found.")
else:
    total_distance_all_clusters = 0

    # Build node -> cluster index map
    node_to_cluster = {}
    for cluster_idx, cluster in enumerate(clusters):
        for node in cluster:
            node_to_cluster[node] = cluster_idx

    cluster_routes = {i: [] for i in range(len(clusters))}
    cluster_served_demand = {i: 0 for i in range(len(clusters))}  # Initialize demand tracker

    for vehicle_id in range(routing.vehicles()):
        index = routing.Start(vehicle_id)
        route = []
        route_demand = 0
        route_distance = 0

        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route.append(node_index)
            route_demand += demands[node_index]
            previous_index = index
            index = solution.Value(routing.NextVar(index))

            if not routing.IsEnd(index):
                from_node = manager.IndexToNode(previous_index)
                to_node = manager.IndexToNode(index)
                route_distance += virtual_distance_matrix[from_node][to_node]

        node_index = manager.IndexToNode(index)
        route.append(node_index)
        from_node = manager.IndexToNode(previous_index)
        to_node = manager.IndexToNode(index)
        route_distance += virtual_distance_matrix[from_node][to_node]

        if len(route) <= 2:  # ignoring vehicles that served no customers
            continue

        # Identify the cluster this vehicle served
        first_customer = next((n for n in route if n != 0), None)
        cluster_id = node_to_cluster.get(first_customer, -1)
        if cluster_id == -1:
            continue

        cluster_routes[cluster_id].append({
            'vehicle_id': vehicle_id,
            'route': route,
            'demand': route_demand,
            'distance': route_distance
        })

        cluster_served_demand[cluster_id] += route_demand
        total_distance_all_clusters += route_distance

    # Printing results per cluster
    for cluster_id, routes in cluster_routes.items():
        if not routes:
            print(f"\n △ Cluster {cluster_id + 1}: No trucks assigned")
            continue

        print(f"\n △ Cluster {cluster_id + 1}:")
        cluster_distance = 0

        for route_info in routes:
            vehicle_id = route_info['vehicle_id']
            route = route_info['route']
            route_demand = route_info['demand']
            route_distance = route_info['distance']

            print(f"\n Truck {vehicle_id + 1} (capacity {vehicles_capacity[0]}):")
            print(f"    ◦ Total Demand: {route_demand}")
            print(f"    ◦ Route Distance: {route_distance} km")
            print("    ◦ Stops:")
            for idx in route:
                node_id = virtual_nodes[idx]['node_id']
                city = virtual_nodes[idx]['city']
                print(f"      • {node_id} ({city})")

            cluster_distance += route_distance

        print(f"\n Total Distance for Cluster {cluster_id + 1}: {cluster_distance} km")

    print(f"\n -----> TOTAL DISTANCE ACROSS ALL CLUSTERS: {total_distance_all_clusters} km")
    print(f"\n -----> TOTAL DEMAND SERVED ACROSS ALL CLUSTERS: {sum(cluster_served_demand.values())}")


### Visualization in QGIS

In [ ]:
import geojson
import time

# Extracting the routes from OR-Tools solution
def extract_routes(manager, routing, solution):
    all_routes = []
    for vehicle_id in range(routing.vehicles()):
        index = routing.Start(vehicle_id)
        route = []
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route.append(node_index)
            index = solution.Value(routing.NextVar(index))
        route.append(manager.IndexToNode(index))  # end depot
        if len(route) > 2:
            all_routes.append(route)
    return all_routes

'''
This part takes a list of coordinates and asks an
external routing (in our case, the API key already used to compute the
real-world distances) to return actual road route geometry.
'''
def get_route_geometry(client, coord_list):
    try:
        time.sleep(1)  # avoid rate limit
        route = client.directions(
            coordinates=coord_list,
            profile='driving-hgv',
            format='geojson',
            radiuses=[1000] * len(coord_list)
        )
        return route['features'][0]['geometry']
    except Exception as e:
        print(f"Error fetching route geometry: {e}")
        return None

# Convert all routes into GeoJSON collection
def routes_to_geojson_with_clusters(clustered_routes, clustered_node_coords, label_prefix, client):
    features = []
    for cluster_idx, routes in enumerate(clustered_routes):
        for route_idx, route in enumerate(routes):
            coords = [clustered_node_coords[cluster_idx][node] for node in route]
            geometry = get_route_geometry(client, coords)
            if geometry:
                feature = geojson.Feature(
                    geometry=geometry,
                    properties={
                        "cluster": cluster_idx + 1,
                        "route_id": f"{label_prefix}_{cluster_idx}_{route_idx}"
                    }
                )
                features.append(feature)
    return geojson.FeatureCollection(features)

clusters = cluster_by_closeness(demands, virtual_distance_matrix, max_cluster_size=9)

routing, manager, solution = solve_clusters(
    clusters,
    virtual_distance_matrix,
    demands,
    vehicles_capacity,
    vehicles,
    use_local_search=True
)

if not solution:
    print("No valid solution found.")
else:
    all_routes = extract_routes(manager, routing, solution)

    # Build node -> cluster map
    node_to_cluster = {}
    for cluster_idx, cluster in enumerate(clusters):
        for node in cluster:
            node_to_cluster[node] = cluster_idx

    # Group routes per cluster
    all_cluster_routes = [[] for _ in range(len(clusters))]
    all_cluster_coords = [{} for _ in range(len(clusters))]

    for route in all_routes:
        first_customer = next((n for n in route if n != 0), None)
        cluster_id = node_to_cluster.get(first_customer, -1)
        if cluster_id == -1:
            continue

        all_cluster_routes[cluster_id].append(route)

        for node in route:
            if node not in all_cluster_coords[cluster_id]:
                all_cluster_coords[cluster_id][node] = (
                    virtual_nodes[node]['x'],
                    virtual_nodes[node]['y']
                )

    geojson_clustered = routes_to_geojson_with_clusters(
        all_cluster_routes,
        all_cluster_coords,
        label_prefix="Cluster",
        client=client
    )

    with open("clustering_by_closeness.geojson", "w") as f:
        geojson.dump(geojson_clustered, f)

## Clustering by demand

### Clustering

In [ ]:
'''
Starting from the highest-demand unvisited node and then
adding other nodes while ensuring that max_cluster_demand is not exceeded
'''

def cluster_by_demand(demands, max_cluster_demand, max_cluster_size=None):

    n = len(demands)
    visited = [False] * n
    visited[0] = True  # no depot
    clusters = []

    while any(not v for v in visited[1:]):
        candidates = [(i, demands[i]) for i in range(1, n) if not visited[i]]
        candidates.sort(key=lambda x: -x[1])
        seed = candidates[0][0]

        cluster = [seed]
        visited[seed] = True
        total_demand = demands[seed]

        # Sorting remaining unvisited nodes (descending order)
        for i, d in candidates[1:]:
            if visited[i]:
                continue
            if max_cluster_size and len(cluster) >= max_cluster_size:
                break
            if max_cluster_demand and total_demand + d > max_cluster_demand:
                continue

            cluster.append(i)
            visited[i] = True
            total_demand += d

        clusters.append(cluster)

    return clusters


### Displaying the solution

In [ ]:
clusters = cluster_by_demand(demands, max_cluster_demand=200, max_cluster_size=None)

routing, manager, solution = solve_clusters(
    clusters,
    virtual_distance_matrix,
    demands,
    vehicles_capacity,
    vehicles,
    use_local_search=True
)

if not solution:
    print("\n No valid solution found.")
else:
    total_distance_all_clusters = 0

    node_to_cluster = {}
    for cluster_idx, cluster in enumerate(clusters):
        for node in cluster:
            node_to_cluster[node] = cluster_idx

    cluster_routes = {i: [] for i in range(len(clusters))}
    cluster_served_demand = {i: 0 for i in range(len(clusters))}

    for vehicle_id in range(routing.vehicles()):
        index = routing.Start(vehicle_id)
        route = []
        route_demand = 0
        route_distance = 0

        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route.append(node_index)
            route_demand += demands[node_index]
            previous_index = index
            index = solution.Value(routing.NextVar(index))

            if not routing.IsEnd(index):
                from_node = manager.IndexToNode(previous_index)
                to_node = manager.IndexToNode(index)
                route_distance += virtual_distance_matrix[from_node][to_node]

        node_index = manager.IndexToNode(index)
        route.append(node_index)
        from_node = manager.IndexToNode(previous_index)
        to_node = manager.IndexToNode(index)
        route_distance += virtual_distance_matrix[from_node][to_node]

        if len(route) <= 2:
            continue

        first_customer = next((n for n in route if n != 0), None)
        cluster_id = node_to_cluster.get(first_customer, -1)
        if cluster_id == -1:
            continue

        cluster_routes[cluster_id].append({
            'vehicle_id': vehicle_id,
            'route': route,
            'demand': route_demand,
            'distance': route_distance
        })

        cluster_served_demand[cluster_id] += route_demand
        total_distance_all_clusters += route_distance

    for cluster_id, routes in cluster_routes.items():
        if not routes:
            print(f"\n △ Cluster {cluster_id + 1}: No trucks assigned")
            continue

        print(f"\n △ Cluster {cluster_id + 1}:")
        cluster_distance = 0

        for route_info in routes:
            vehicle_id = route_info['vehicle_id']
            route = route_info['route']
            route_demand = route_info['demand']
            route_distance = route_info['distance']

            print(f"\n Truck {vehicle_id + 1} (capacity {vehicles_capacity[0]}):")
            print(f"    ◦ Total Demand: {route_demand}")
            print(f"    ◦ Route Distance: {route_distance} km")
            print("    ◦ Stops:")
            for idx in route:
                node_id = virtual_nodes[idx]['node_id']
                city = virtual_nodes[idx]['city']
                print(f"      • {node_id} ({city})")

            cluster_distance += route_distance

        print(f"\n Total Distance for Cluster {cluster_id + 1}: {cluster_distance} km")

    print(f"\n -----> TOTAL DISTANCE ACROSS ALL CLUSTERS: {total_distance_all_clusters} km")
    print(f"\n -----> TOTAL DEMAND SERVED ACROSS ALL CLUSTERS: {sum(cluster_served_demand.values())}")


### Visualization in QGIS

In [ ]:
def extract_routes(manager, routing, solution):
    all_routes = []
    for vehicle_id in range(routing.vehicles()):
        index = routing.Start(vehicle_id)
        route = []
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route.append(node_index)
            index = solution.Value(routing.NextVar(index))
        route.append(manager.IndexToNode(index))  # end depot
        if len(route) > 2:
            all_routes.append(route)
    return all_routes

def get_route_geometry(client, coord_list):
    try:
        time.sleep(1)  # avoid rate limit
        radiuses = [1000.0] * len(coord_list)
        route = client.directions(
            coordinates=coord_list,
            profile='driving-hgv',
            format='geojson',
            radiuses=[1000] * len(coord_list)
        )
        return route['features'][0]['geometry']
    except Exception as e:
        print(f"Error fetching route geometry: {e}")
        return None

def routes_to_geojson_with_clusters(clustered_routes, clustered_node_coords, label_prefix, client):
    features = []
    for cluster_idx, routes in enumerate(clustered_routes):
        for route_idx, route in enumerate(routes):
            coords = [clustered_node_coords[cluster_idx][node] for node in route]
            geometry = get_route_geometry(client, coords)
            if geometry:
                feature = geojson.Feature(
                    geometry=geometry,
                    properties={
                        "cluster": cluster_idx + 1,
                        "route_id": f"{label_prefix}_{cluster_idx}_{route_idx}"
                    }
                )
                features.append(feature)
    return geojson.FeatureCollection(features)

clusters = cluster_by_demand(demands, max_cluster_demand=180, max_cluster_size=None)

routing, manager, solution = solve_clusters(
    clusters,
    virtual_distance_matrix,
    demands,
    vehicles_capacity,
    vehicles,
    use_local_search=True
)

if not solution:
    print("No valid solution found.")
else:
    all_routes = extract_routes(manager, routing, solution)

    # Build node -> cluster map
    node_to_cluster = {}
    for cluster_idx, cluster in enumerate(clusters):
        for node in cluster:
            node_to_cluster[node] = cluster_idx

    # Group routes per cluster
    all_cluster_routes = [[] for _ in range(len(clusters))]
    all_cluster_coords = [{} for _ in range(len(clusters))]

    for route in all_routes:
        first_customer = next((n for n in route if n != 0), None)
        cluster_id = node_to_cluster.get(first_customer, -1)
        if cluster_id == -1:
            continue

        all_cluster_routes[cluster_id].append(route)

        for node in route:
            if node not in all_cluster_coords[cluster_id]:
                all_cluster_coords[cluster_id][node] = (
                    virtual_nodes[node]['x'],
                    virtual_nodes[node]['y']
                )

    geojson_clustered = routes_to_geojson_with_clusters(
        all_cluster_routes,
        all_cluster_coords,
        label_prefix="Cluster",
        client=client
    )

    with open("clustering_by_demand.geojson", "w") as f:
        geojson.dump(geojson_clustered, f)

